# Amazon SageMaker Model Monitor
This notebook shows how to:
* Host a machine learning model in Amazon SageMaker and capture inference requests, results, and metadata 
* Analyze a training dataset to generate baseline constraints
* Monitor a live endpoint for violations against constraints

---
## Background

Amazon SageMaker provides every developer and data scientist with the ability to build, train, and deploy machine learning models quickly. Amazon SageMaker is a fully-managed service that encompasses the entire machine learning workflow. You can label and prepare your data, choose an algorithm, train a model, and then tune and optimize it for deployment. You can deploy your models to production with Amazon SageMaker to make predictions and lower costs than was previously possible.

In addition, Amazon SageMaker enables you to capture the input, output and metadata for invocations of the models that you deploy. It also enables you to analyze the data and monitor its quality. In this notebook, you learn how Amazon SageMaker enables these capabilities.

---
## Setup

To get started, make sure you have these prerequisites completed.

* Specify an AWS Region to host your model.
* An IAM role ARN exists that is used to give Amazon SageMaker access to your data in Amazon Simple Storage Service (Amazon S3). See the documentation for how to fine tune the permissions needed. 
* Create an S3 bucket used to store the data used to train your model, any additional model data, and the data captured from model invocations. For demonstration purposes, you are using the same bucket for these. In reality, you might want to separate them with different security policies.

In [1]:
import boto3
import os
import sagemaker
from sagemaker import get_execution_role

region = boto3.Session().region_name
role = get_execution_role()
sess = sagemaker.session.Session()
bucket = sess.default_bucket() 
prefix = 'tf-2-workflow'

s3_capture_upload_path = 's3://{}/{}/monitoring/datacapture'.format(bucket, prefix)

reports_prefix = '{}/reports'.format(prefix)
s3_report_path = 's3://{}/{}'.format(bucket,reports_prefix)

print("Capture path: {}".format(s3_capture_upload_path))
print("Report path: {}".format(s3_report_path))

Capture path: s3://sagemaker-eu-central-1-022505235570/tf-2-workflow/monitoring/datacapture
Report path: s3://sagemaker-eu-central-1-022505235570/tf-2-workflow/reports


# PART A: Capturing real-time inference data from Amazon SageMaker endpoints
Create an endpoint to showcase the data capture capability in action.


### Deploy the model to Amazon SageMaker
Start with deploying the trained TensorFlow model from lab 03.

In [2]:
import boto3

def get_latest_training_job_name(base_job_name):
    client = boto3.client('sagemaker')
    response = client.list_training_jobs(NameContains=base_job_name, SortBy='CreationTime', 
                                         SortOrder='Descending', StatusEquals='Completed')
    if len(response['TrainingJobSummaries']) > 0 :
        return response['TrainingJobSummaries'][0]['TrainingJobName']
    else:
        raise Exception('Training job not found.')

def get_training_job_s3_model_artifacts(job_name):
    client = boto3.client('sagemaker')
    response = client.describe_training_job(TrainingJobName=job_name)
    s3_model_artifacts = response['ModelArtifacts']['S3ModelArtifacts']
    return s3_model_artifacts

latest_training_job_name = get_latest_training_job_name('tf-2-workflow')
print(latest_training_job_name)
model_path = get_training_job_s3_model_artifacts(latest_training_job_name)
print(model_path)

tf-2-workflow-22-09-23-03-015-763adf42
s3://sagemaker-eu-central-1-022505235570/tf-2-workflow-22-09-23-03-015-763adf42/output/model.tar.gz


Here, you create the model object with the image and model data.

In [3]:
from sagemaker.tensorflow.model import TensorFlowModel

tensorflow_model = TensorFlowModel(
    model_data = model_path,
    role = role,
    framework_version = '2.3.1'
)


In [4]:
from time import gmtime, strftime
from sagemaker.model_monitor import DataCaptureConfig

endpoint_name = 'tf-2-workflow-endpoint-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_name)

predictor = tensorflow_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m5.xlarge',
    endpoint_name=endpoint_name,
    data_capture_config=DataCaptureConfig(
        enable_capture=True,
        sampling_percentage=100,
        destination_s3_uri=s3_capture_upload_path
    )
)

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


tf-2-workflow-endpoint-2021-03-22-10-41-03
-----------!

### Prepare dataset

Next, we'll import the dataset. The dataset itself is small and relatively issue-free. For example, there are no missing values, a common problem for many other datasets. Accordingly, preprocessing just involves normalizing the data.

In [5]:
import numpy as np
from tensorflow.python.keras.datasets import boston_housing
from sklearn.preprocessing import StandardScaler

(x_train, y_train), (x_test, y_test) = boston_housing.load_data()
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

## Invoke the deployed model

You can now send data to this endpoint to get inferences in real time. Because you enabled the data capture in the previous steps, the request and response payload, along with some additional metadata, is saved in the Amazon Simple Storage Service (Amazon S3) location you have specified in the DataCaptureConfig.

This step invokes the endpoint with included sample data for about 3 minutes. Data is captured based on the sampling percentage specified and the capture continues until the data capture option is turned off.

In [6]:
%%time 

import time

print("Sending test traffic to the endpoint {}. \nPlease wait...".format(endpoint_name))

flat_list =[]
for item in x_test:
    result = predictor.predict(item)['predictions'] 
    flat_list.append(float('%.1f'%(np.array(result))))
    time.sleep(1.8)
    
print("Done!")
print('predictions: \t{}'.format(np.array(flat_list)))

Sending test traffic to the endpoint tf-2-workflow-endpoint-2021-03-22-10-41-03. 
Please wait...
Done!
predictions: 	[12.7 19.  21.5 30.4 22.1 21.  28.5 24.8 18.9 19.4 16.9 18.7 17.2 37.
 13.6 23.6 24.  24.1 15.4 19.5 12.7 12.9 21.4 17.6 24.8 25.6 30.6 24.6
 12.7 23.7 21.3 13.  34.1 23.1 17.1 12.7 18.9 19.9 22.1 25.7 24.2 26.3
 15.4 37.  29.1 21.1 27.4 19.  19.5 21.8 29.2 19.7 12.7 15.  36.2 25.1
 15.4 37.1 35.6 21.6 23.9 17.3 15.3 19.7 22.2 25.1 15.2 25.7 12.7 12.7
 19.1 25.9 22.3 13.1 26.2 19.5 22.9 22.  31.6 12.7 21.6 37.  19.8 16.9
 19.5 18.4 17.4 20.3 22.3 33.6 18.6 24.5 25.5 31.3 32.3 21.1 36.7 35.
 24.3 37.  32.3 22.4]
CPU times: user 302 ms, sys: 31.2 ms, total: 333 ms
Wall time: 3min 5s


## View captured data

Now list the data capture files stored in Amazon S3. You should expect to see different files from different time periods organized based on the hour in which the invocation occurred. The format of the Amazon S3 path is:

`s3://{destination-bucket-prefix}/{endpoint-name}/{variant-name}/yyyy/mm/dd/hh/filename.jsonl`

<b>Note that the delivery of capture data to Amazon S3 can require a couple of minutes so next cell might error. If this happens, please retry after a minute.</b>

In [7]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix='tf-2-workflow/monitoring/datacapture/')
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\n ".join(capture_files))

Found Capture Files:
tf-2-workflow/monitoring/datacapture/tf-2-workflow-endpoint-2021-03-22-10-41-03/AllTraffic/2021/03/22/10/57-47-993-8069436d-6535-4e5d-be17-b48b52e50cbb.jsonl
 tf-2-workflow/monitoring/datacapture/tf-2-workflow-endpoint-2021-03-22-10-41-03/AllTraffic/2021/03/22/10/58-48-150-ba44bc72-0a55-4d32-a4fd-dab67f8bf28f.jsonl


Next, view the contents of a single capture file. Here you should see all the data captured in an Amazon SageMaker specific JSON-line formatted file. Take a quick peek at the first few lines in the captured file.

In [8]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get('Body').read().decode("utf-8")

capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"application/json","mode":"INPUT","data":"[-0.3846146658753033, -0.4836154708652843, -0.04328034837026877, -0.2568327484687563, -1.2322543225547462, -0.031148842501293867, -2.2507883885350743, 0.7629475894761004, -0.6262490526587586, -0.6072060061360174, 0.32944628645426327, 0.2381714623786403, -0.7177935480963249]","encoding":"JSON"},"endpointOutput":{"observedContentType":"application/json","mode":"OUTPUT","data":"{\n    \"predictions\": [[23.0875893]\n    ]\n}","encoding":"JSON"}},"eventMetadata":{"eventId":"72c01046-9516-414c-8456-af55d0f9003b","inferenceTime":"2021-03-22T10:58:48Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"application/json","mode":"INPUT","data":"[1.0109107738040224, -0.4836154708652843, 1.0283257954396188, -0.2568327484687563, 0.19329471283500554, -0.7815981666935935, -0.4411458316241723, -0.452024740593543, 1.6758857724016463, 1.5652874992218142, 0.7844763709927688, 0.44807

Finally, the contents of a single line is present below in a formatted JSON file so that you can observe a little better.

In [9]:
import json
print(json.dumps(json.loads(capture_file.split('\n')[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "application/json",
      "mode": "INPUT",
      "data": "[-0.3846146658753033, -0.4836154708652843, -0.04328034837026877, -0.2568327484687563, -1.2322543225547462, -0.031148842501293867, -2.2507883885350743, 0.7629475894761004, -0.6262490526587586, -0.6072060061360174, 0.32944628645426327, 0.2381714623786403, -0.7177935480963249]",
      "encoding": "JSON"
    },
    "endpointOutput": {
      "observedContentType": "application/json",
      "mode": "OUTPUT",
      "data": "{\n    \"predictions\": [[23.0875893]\n    ]\n}",
      "encoding": "JSON"
    }
  },
  "eventMetadata": {
    "eventId": "72c01046-9516-414c-8456-af55d0f9003b",
    "inferenceTime": "2021-03-22T10:58:48Z"
  },
  "eventVersion": "0"
}


As you can see, each inference request is captured in one line in the jsonl file. The line contains both the input and output merged together. In the example, you provided the ContentType as `text/csv` which is reflected in the `observedContentType` value. Also, you expose the encoding that you used to encode the input and output payloads in the capture format with the `encoding` value.

To recap, you observed how you can enable capturing the input or output payloads to an endpoint with a new parameter. You have also observed what the captured format looks like in Amazon S3. Next, continue to explore how Amazon SageMaker helps with monitoring the data collected in Amazon S3.

# PART B: Model Monitor - Baselining and continuous monitoring

In addition to collecting the data, Amazon SageMaker provides the capability for you to monitor and evaluate the data observed by the endpoints. For this:
1. Create a baseline with which you compare the realtime traffic. 
1. Once a baseline is ready, setup a schedule to continously evaluate and compare against the baseline.

## 1. Constraint suggestion with baseline/training dataset

The training dataset with which you trained the model is usually a good baseline dataset. Note that the training dataset data schema and the inference dataset schema should exactly match (i.e. the number and order of the features).

From the training dataset you can ask Amazon SageMaker to suggest a set of baseline `constraints` and generate descriptive `statistics` to explore the data. For this example, upload the training dataset that was used to train the pre-trained model included in this example. If you already have it in Amazon S3, you can directly point to it.

### Prepare training dataset with headers

In [ ]:
import pandas as pd
dt = pd.DataFrame(data = x_train, 
                  columns = ["CRIM", "ZN", "INDUS", "CHAS","NOX","RM","AGE","DIS","RAD","TAX","PTRATIO","B","LSTAT"])

dt.to_csv("training-dataset-with-header.csv", index = False)

In [ ]:
# copy over the training dataset to Amazon S3 (if you already have it in Amazon S3, you could reuse it)
baseline_prefix = prefix + '/baselining'
baseline_data_prefix = baseline_prefix + '/data'
baseline_results_prefix = baseline_prefix + '/results'

baseline_data_uri = 's3://{}/{}'.format(bucket,baseline_data_prefix)
baseline_results_uri = 's3://{}/{}'.format(bucket, baseline_results_prefix)
print('Baseline data uri: {}'.format(baseline_data_uri))
print('Baseline results uri: {}'.format(baseline_results_uri))


In [ ]:
training_data_file = open("training-dataset-with-header.csv", 'rb')
s3_key = os.path.join(baseline_prefix, 'data', 'training-dataset-with-header.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(s3_key).upload_fileobj(training_data_file)

### Create a baselining job with training dataset

Now that you have the training data ready in Amazon S3, start a job to `suggest` constraints. `DefaultModelMonitor.suggest_baseline(..)` starts a `ProcessingJob` using an Amazon SageMaker provided Model Monitor container to generate the constraints.

In [ ]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri+'/training-dataset-with-header.csv',
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)

### Explore the generated constraints and statistics

In [ ]:
s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=bucket, Prefix=baseline_results_prefix)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Files:")
print("\n ".join(report_files))

In [ ]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

In [ ]:
constraints_df = pd.io.json.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

## 2. Analyzing collected data for data quality issues

### Create a schedule

You can create a model monitoring schedule for the endpoint created earlier. Use the baseline resources (constraints and statistics) to compare against the realtime traffic.

From the analysis above, you saw how the captured data is saved - that is the standard input and output format for Tensorflow models. But Model Monitor is framework-agnostic, and expects a specific format [explained in the docs](https://docs.aws.amazon.com/sagemaker/latest/dg/model-monitor-pre-and-post-processing.html#model-monitor-pre-processing-script):
- Input
    - Flattened JSON `{"feature0": <value>, "feature1": <value>...}`
    - Tabular `"<value>, <value>..."`
- Output:
    - Flattened JSON `{"prediction0": <value>, "prediction1": <value>...}`
    - Tabular `"<value>, <value>..."`
    
We need to transform the input records to comply with this requirement. Model Monitor offers _pre-processing scripts_ in Python to transform the input. The cell below has the script that will work for our case.

In [ ]:
%%writefile preprocessing.py

import json

def preprocess_handler(inference_record):
    input_data = json.loads(inference_record.endpoint_input.data)
    input_data = {f"feature{i}": val for i, val in enumerate(input_data)}
    
    output_data = json.loads(inference_record.endpoint_output.data)["predictions"][0][0]
    output_data = {"prediction0": output_data}
    
    return{**input_data}

We'll upload this script to an s3 destination and pass it as the `record_preprocessor_script` parameter to the `create_monitoring_schedule` call.

In [ ]:
script_s3_dest_path = f"s3://{bucket}/{prefix}/artifacts/modelmonitor"
script_s3_dest = sagemaker.s3.S3Uploader.upload("preprocessing.py", script_s3_dest_path)
print(script_s3_dest)

In [ ]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime

mon_schedule_name = 'DEMO-tf-2-workflow-model-monitor-schedule-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=mon_schedule_name,
    endpoint_input=predictor.endpoint,
    record_preprocessor_script=script_s3_dest,
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

### Generating violations artificially

In order to get some result relevant to monitoring analysis, you can try and generate artificially some inferences with feature values causing specific violations, and then invoke the endpoint with this data

Looking at our RM and AGE features:

- RM - average number of rooms per dwelling
- AGE - proportion of owner-occupied units built prior to 1940

Let's simulate a situation where the average number of rooms is 0, and proportion of owner-occupied units built is 1000.

In [ ]:
df_with_violations = pd.read_csv("training-dataset-with-header.csv")
df_with_violations["RM"] = 0
df_with_violations["AGE"] = 1000
df_with_violations

### Start generating some artificial traffic
The cell below starts a thread to send some traffic to the endpoint. Note that you need to stop the kernel to terminate this thread. If there is no traffic, the monitoring jobs are marked as `Failed` since there is no data to process.

In [ ]:
from threading import Thread
from time import sleep
import time

def invoke_endpoint():
    for item in df_with_violations.to_numpy():
        result = predictor.predict(item)['predictions'] 
        time.sleep(1)

def invoke_endpoint_forever():
    while True:
        invoke_endpoint()
        
thread = Thread(target = invoke_endpoint_forever)
thread.start()

# Note that you need to stop the kernel to stop the invocations

### Describe and inspect the schedule
Once you describe, observe that the MonitoringScheduleStatus changes to Scheduled.

In [ ]:
desc_schedule_result = my_default_monitor.describe_schedule()
print('Schedule status: {}'.format(desc_schedule_result['MonitoringScheduleStatus']))

### List executions
The schedule starts jobs at the previously specified intervals. Here, you list the latest five executions. Note that if you are kicking this off after creating the hourly schedule, you might find the executions empty. You might have to wait until you cross the hour boundary (in UTC) to see executions kick off. The code below has the logic for waiting.

Note: Even for an hourly schedule, Amazon SageMaker has a buffer period of 20 minutes to schedule your execution. You might see your execution start in anywhere from zero to ~20 minutes from the hour boundary. This is expected and done for load balancing in the backend.

In [ ]:
mon_executions = my_default_monitor.list_executions()
print("We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.\nWe will have to wait till we hit the hour...")

while len(mon_executions) == 0:
    print("Waiting for the 1st execution to happen...")
    time.sleep(60)
    mon_executions = my_default_monitor.list_executions()    

### Inspect a specific execution (latest execution)
In the previous cell, you picked up the latest completed or failed scheduled execution. Here are the possible terminal states and what each of them mean: 
* Completed - This means the monitoring execution completed and no issues were found in the violations report.
* CompletedWithViolations - This means the execution completed, but constraint violations were detected.
* Failed - The monitoring execution failed, maybe due to client error (perhaps incorrect role premissions) or infrastructure issues. Further examination of FailureReason and ExitMessage is necessary to identify what exactly happened.
* Stopped - job exceeded max runtime or was manually stopped.

In [ ]:
latest_execution = mon_executions[-1] # latest execution's index is -1, second to last is -2 and so on..
#time.sleep(60)
latest_execution.wait(logs=False)

print("Latest execution status: {}".format(latest_execution.describe()['ProcessingJobStatus']))
print("Latest execution result: {}".format(latest_execution.describe()['ExitMessage']))

latest_job = latest_execution.describe()
if (latest_job['ProcessingJobStatus'] != 'Completed'):
        print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")

In [ ]:
report_uri=latest_execution.output.destination
print('Report Uri: {}'.format(report_uri))

### List the generated reports

In [ ]:
from urllib.parse import urlparse
s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip('/')
print('Report bucket: {}'.format(report_bucket))
print('Report key: {}'.format(report_key))

s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Report Files:")
print("\n ".join(report_files))

### Violations report

If there are any violations compared to the baseline, they will be listed here.

In [ ]:
violations = my_default_monitor.latest_monitoring_constraint_violations()
pd.set_option('display.max_colwidth', -1)
constraints_df = pd.io.json.json_normalize(violations.body_dict["violations"])
constraints_df.head(10)

## Delete the resources

You can keep your endpoint running to continue capturing data. If you do not plan to collect more data or use this endpoint further, you should delete the endpoint to avoid incurring additional charges. Note that deleting your endpoint does not delete the data that was captured during the model invocations. That data persists in Amazon S3 until you delete it yourself.

But before that, you need to delete the schedule first.

In [ ]:
my_default_monitor.delete_monitoring_schedule()
time.sleep(120) # actually wait for the deletion

In [ ]:
predictor.delete_endpoint()